### Imports and Utility Variables

In [1]:
%run ../imports.py

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
foul_types_raw_all = ['Personal','P.FOUL',
                      'Turnover: Shot Clock',
                      'S.FOUL','Shooting',
                      'L.B.FOUL',
                      'OFF.Foul',
                      'Traveling']

foul_types = ['Personal', '24 Second', 'Shooting', 'Loose Ball', 'Offensive', 'Traveling']

# Originally included more foul types, but processing a season's data took more than 4 hours
foul_types_dict = {'P.FOUL':'Personal', # 'Personal Take Foul':'Personal',
                   'Personal':'Personal', # 'Personal Block':'Personal',
                   'Turnover: Shot Clock':'24 Second',
                   'S.FOUL':'Shooting','Shooting':'Shooting',
                   'L.B.FOUL':'Loose Ball',
                   'OFF.Foul':'Offensive',
                   'Traveling':'Traveling'
                   #'T.Foul':'Technical','T.Foul (Def. 3 Sec':'Defensive 3 Second'
}

features = ['SCORE','HOMEDESCRIPTION','VISITORDESCRIPTION','EVENTNUM','GAME_ID','PCTIMESTRING',
            'PERIOD','PLAYER1_ID','PLAYER1_NAME','PLAYER1_TEAM_NICKNAME',
            'PLAYER2_ID','PLAYER2_NAME','PLAYER2_TEAM_NICKNAME',
            'PLAYER3_ID','PLAYER3_NAME','PLAYER3_TEAM_NICKNAME',
            'SCOREMARGIN'
           ]

team_to_abrev_dict = {'76ers':'PHI',
'Bucks':'MIL',
'Bulls':'CHI',
'Cavaliers':'CLE',
'Celtics':'BOS',
'Clippers':'LAC',
'Grizzlies':'MEM',
'Hawks':'ATL',
'Heat':'MIA',
'Hornets':'CHA',
'Jazz':'UTA',
'Kings':'SAC',
'Knicks':'NYK',
'Lakers':'LAL',
'Magic':'ORL',
'Mavericks':'DAL',
'Nets':'BKN',
'Nuggets':'DEN',
'Pacers':'IND',
'Pelicans':'NOP',
'Pistons':'DET',
'Raptors':'TOR',
'Rockets':'HOU',
'Spurs':'SAS',
'Suns':'PHX',
'Thunder':'OKC',
'Timberwolves':'MIN',
'Trail Blazers':'POR',
'Warriors':'GSW',
'Wizards':'WAS'}

In [3]:
df = pd.read_csv('./data/play_by_play.csv', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135698 entries, 0 to 1135697
Data columns (total 30 columns):
SCORE                    295258 non-null object
HOMEDESCRIPTION          587421 non-null object
VISITORDESCRIPTION       580417 non-null object
EVENTNUM                 1135698 non-null int64
GAME_ID                  1135698 non-null int64
PCTIMESTRING             1135698 non-null object
PERIOD                   1135698 non-null int64
PLAYER1_ID               1135698 non-null int64
PLAYER1_NAME             1034876 non-null object
PLAYER2_ID               1135698 non-null int64
PLAYER2_NAME             361816 non-null object
PLAYER3_ID               1135698 non-null int64
PLAYER3_NAME             27941 non-null object
SCOREMARGIN              295258 non-null object
Foul                     102457 non-null object
Fouler                   99820 non-null object
Fouler_team              99820 non-null object
is_home_team             102342 non-null object
Foulee                  

In [4]:
df_usage = pd.read_csv('./data/usage.csv', index_col='Unnamed: 0', compression='gzip')
df_usage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2460 entries, 21500758 to 21600130
Columns: 2837 entries, Jabari Parker_AdjUsage to Score_label
dtypes: float64(2836), object(1)
memory usage: 53.3+ MB


In [305]:
df_l2m = pd.read_csv('./data/L2M_final.csv')

#### Fixing Last 2 Minute Score Labels (acts as the Game ID column)

In [306]:
df_l2m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2902 entries, 0 to 2901
Data columns (total 17 columns):
Unnamed: 0     2902 non-null int64
Period         2902 non-null object
data           2902 non-null object
Fouler_team    2902 non-null object
Fouler         2902 non-null object
Foulee         2902 non-null object
Review         2881 non-null object
Date           2902 non-null object
Unnamed: 8     96 non-null float64
Score          2902 non-null object
Soup           2902 non-null object
Time           2902 non-null object
Comment        2764 non-null object
Foul           2902 non-null object
Away_team      2902 non-null object
Home_team      2902 non-null object
Score_label    2902 non-null object
dtypes: float64(1), int64(1), object(15)
memory usage: 385.5+ KB


In [320]:
def l2m_home_away(score_text):
    t = score_text.split('_')
    team1 = t[0]
    return team1

def l2m_home_away2(score_text):
    t = score_text.split('_')
    team2 = t[1].split('(')[0] # to get rid of overtime label if it has it, if not still works
    return team2

def correct_score_label(row):
    if (row['Score_label1'] in all_games) & (row['Score_label2'] in all_games):
        return 'error'
    if row['Score_label1'] in all_games:
        return row['Score_label1']
    elif row['Score_label2'] in all_games:
        return row['Score_label2']
    else:
        return 'error'

def get_game_id(score_label):
    try: return groupby_score.loc[score_label, 'game_id']
    except: return 'error'

In [244]:
groupby_score = df[['GAME_ID','Home_score','Away_score','Score_label']].groupby('GAME_ID').last()
groupby_score.head()
# Play-by-play home/away was switched, fixing below

,Home_score,Away_score,Score_label
GAME_ID,,,
21500001,94,106,Pistons94_Hawks106
21500002,97,95,Cavaliers97_Bulls95
21500003,111,95,Pelicans111_Warriors95
21500004,87,88,Wizards87_Magic88
21500005,112,95,76ers112_Celtics95


In [239]:
def make_score_cols(df):
    df['Home_score'] = df['SCORE'].map(home_score)
    df['Away_score'] = df['SCORE'].map(away_score)
    
def home_score(score_text):
    try:
        lis = score_text.split(' ') # example: 97 - 95 -> ['97', '-', '95']
        return int(lis[2])
    except: return 0

def away_score(score_text):
    try:
        lis = score_text.split(' ')
        return int(lis[0])
    except: return 0
    
def make_score_label(df):
    df['Score_label'] = df['GAME_ID'].map(score_label)

temp = df[['GAME_ID','Home_team','Away_team']].groupby('GAME_ID').first()
    
def score_label(game_id):
    str1 = str(temp.loc[game_id, 'Away_team']) + str(groupby_score.loc[game_id, 'Away_score'])
    str2 = str(temp.loc[game_id, 'Home_team']) + str(groupby_score.loc[game_id, 'Home_score'])
    output = str1 + '_' + str2
    return output.replace(' ', '')

In [234]:
make_score_cols(df)

In [235]:
df['Score_label_error'] = df['Score_label']

In [245]:
make_score_label(df)

In [246]:
(df['Score_label_error'] == df['Score_label']).sum()

0

In [247]:
df.head()

,SCORE,HOMEDESCRIPTION,VISITORDESCRIPTION,EVENTNUM,GAME_ID,PCTIMESTRING,PERIOD,PLAYER1_ID,PLAYER1_NAME,PLAYER2_ID,PLAYER2_NAME,PLAYER3_ID,PLAYER3_NAME,SCOREMARGIN,Foul,Fouler,Fouler_team,is_home_team,Foulee,Foulee_team,PLAYER1_TEAM_NICKNAME,PLAYER2_TEAM_NICKNAME,PLAYER3_TEAM_NICKNAME,Home_score,Away_score,Minute,Second,Home_team,Away_team,Score_label,Score_label_error
0,NaN,NaN,NaN,0,21500758,12:00,1,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,12,0,Jazz,Bucks,Bucks81_Jazz84,Bucks84_Jazz81
1,NaN,Jump Ball Gobert vs. Monroe: Tip to Middleton,NaN,1,21500758,12:00,1,203497,Rudy Gobert,202328,Greg Monroe,203114,Khris Middleton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,Bucks,Bucks,0,0,12,0,Jazz,Bucks,Bucks81_Jazz84,Bucks84_Jazz81
2,NaN,NaN,MISS Parker 17' Pullup Jump Shot,2,21500758,11:45,1,203953,Jabari Parker,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bucks,NaN,NaN,0,0,11,45,Jazz,Bucks,Bucks81_Jazz84,Bucks84_Jazz81
3,NaN,Neto REBOUND (Off:0 Def:1),NaN,3,21500758,11:44,1,203526,Raul Neto,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,NaN,NaN,0,0,11,44,Jazz,Bucks,Bucks81_Jazz84,Bucks84_Jazz81
4,0 - 2,Neto 4' Fadeaway Jumper (2 PTS),NaN,4,21500758,11:19,1,203526,Raul Neto,0,NaN,0,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,NaN,NaN,2,0,11,19,Jazz,Bucks,Bucks81_Jazz84,Bucks84_Jazz81


In [249]:
df.drop('Score_label_error', axis=1, inplace=True)

In [317]:
groupby_score['game_id'] = groupby_score.index
groupby_score.set_index('Score_label', drop=False, inplace=True)
groupby_score.head()

,Home_score,Away_score,Score_label,game_id
Score_label,,,,
Pistons106_Hawks94,94,106,Pistons106_Hawks94,21500001
Cavaliers95_Bulls97,97,95,Cavaliers95_Bulls97,21500002
Pelicans95_Warriors111,111,95,Pelicans95_Warriors111,21500003
Wizards88_Magic87,87,88,Wizards88_Magic87,21500004
76ers95_Celtics112,112,95,76ers95_Celtics112,21500005


In [326]:
df_l2m['game_id'] = df_l2m['Score_label'].map(get_game_id)

In [307]:
df_l2m['team1'] = df_l2m['Score'].map(l2m_home_away)
df_l2m['team2'] = df_l2m['Score'].map(l2m_home_away2)
df_l2m['Score_label1'] = df_l2m['team1'] + '_' + df_l2m['team2']
df_l2m['Score_label2'] = df_l2m['team2'] + '_' + df_l2m['team1']

In [328]:
df_l2m[df_l2m['game_id']=='error'].shape

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\pandas\core\ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


(151, 22)

In [324]:
df_l2m['Score_label'] = df_l2m.apply(correct_score_label, axis=1)
df_l2m[df_l2m['Score_label']=='error'].shape

(151, 22)

In [310]:
all_games = set(df['Score_label'])
len(all_games)

2458

In [286]:
test = df[df['Score_label'].str.contains('Warriors118')]['Score_label'].sort_values(ascending=False).unique()
test

array(['Warriors118_Nuggets105', 'Warriors118_Magic98',
       'Warriors118_Heat112'], dtype=object)

In [331]:
# Manually fixing last game_ids, about 30 errors still
df_l2m = pd.read_csv('./data/L2M_final.csv')

In [260]:
df.to_csv('./data/play_by_play.csv', index=False, compression='gzip')

In [329]:
df_l2m.to_csv('./data/L2M_final.csv')

#### Making Right Index for df_usage

In [167]:
df.GAME_ID.unique() # luckily, game_ids are stored in same order in main play-by-play dataset
# These are needed as the index of df_usage

array([21500758, 21500991, 21500538, ..., 21600255, 21600289, 21600130],
      dtype=int64)

In [168]:
len(df.GAME_ID.unique())

2460

In [170]:
df_usage.index = df.GAME_ID.unique()
df_usage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2460 entries, 21500758 to 21600130
Columns: 2464 entries, Jabari Parker_AdjUsage to Joe Harris_5-10ft
dtypes: float64(2464)
memory usage: 46.3 MB


In [171]:
df_usage.to_csv('./data/usage.csv', index=True, compression='gzip')

In [12]:
df_l2m[~df_l2m['Unnamed: 11'].isnull()]

,Period,data,Fouler,Foulee,Review,Date,Score,Soup,Time,Comment,Foul,Unnamed: 11
6502,Q4,00:15.1 Foul: Shooting,Patrick Beverley,Evan Fournier,CNC,0015-12-23,Magic104_Rockets101,Q4 00:15.1 Foul: Shooting Patrick Beverley Eva...,00:15.1,LHH shows Beverley (HOU) maintains a legal gua...,Beverley is permitted to apply a forearm to F...,Shooting
6944,Q4,00:27.8 Foul: Loose Ball,Otto Porter,Andre Drummond,CNC,0015-11-21,Wizards97_Pistons95,Q4 00:27.8 Foul: Loose Ball Otto Porter Andre ...,00:27.8,Porter (WAS) does not dislodge Drummond (DET) ...,both players maintain legal positions as they...,Loose Ball


In [11]:
df_l2m.loc[6502, 'Comment'] = df_l2m.loc[6502, 'Comment'] + df_l2m.loc[6502, 'Foul']
df_l2m.loc[6944, 'Comment'] = df_l2m.loc[6944, 'Comment'] + df_l2m.loc[6944, 'Foul']

In [13]:
df_l2m.loc[6502, 'Comment']

"LHH shows Beverley (HOU) maintains a legal guarding position during Fournier's (ORL) drive Beverley is permitted to apply a forearm to Fournier in the LBD to"

In [14]:
df_l2m.loc[6502, 'Foul'] = df_l2m.loc[6502, 'Unnamed: 11']
df_l2m.loc[6944, 'Foul'] = df_l2m.loc[6944, 'Unnamed: 11']

In [16]:
df_l2m.drop('Unnamed: 11', axis=1, inplace=True)

In [18]:
df_l2m.to_csv('./data/L2M_processed.csv', index=False, compression='gzip')

### Final Data Munging

In [112]:
def convert_l2m_score_away(text):
    try:
        match = re.search('([76]*[A-z]+)\d+_([76]*[A-z]+)\d+', text)
        return match.group(1)
    except: return ''
    
def convert_l2m_score_home(text):
    try:
        match = re.search('([76]*[A-z]+)\d+_([76]*[A-z]+)\d+', text)
        return match.group(2)
    except: return ''

In [55]:
match = re.search('([76]*[A-z]+)\d+_([76]*[A-z]+)\d+', 'Warriors118_76ers113')
match.group(2)

'76ers'

In [128]:
df_l2m['Away_team'] = df_l2m['Score'].map(convert_l2m_score_away)
df_l2m['Home_team'] = df_l2m['Score'].map(convert_l2m_score_home)
df_l2m.tail()

,Period,data,Fouler,Foulee,Review,Date,Score,Soup,Time,Comment,Foul,Away_team,Home_team
7243,Q4,00:34.9 Foul: Shooting,Nikola Vucevic,Otto Porter,CNC,0015-10-28,Wizards88_Magic87,Q4 00:34.9 Foul: Shooting Nikola Vucevic Otto ...,00:34.9,RATR shows Vucevic (ORL) makes incidental cont...,Shooting,Wizards,Magic
7244,Q4,00:19.6 Violation: Delay of Game,NaN,NaN,CC,0015-10-28,Wizards88_Magic87,Q4 00:19.6 Violation: Delay of Game nan nan CC...,00:19.6,WAS is not ready to resume play when the time-...,NaN,Wizards,Magic
7245,Q4,00:10.3 Foul: Personal,John Wall,Tobias Harris,CC,0015-10-28,Wizards88_Magic87,Q4 00:10.3 Foul: Personal John Wall Tobias Har...,00:10.3,Wall (WAS) commits a take foul on Harris (ORL).,Personal,Wizards,Magic
7246,Q4,00:03.5 Instant Replay: Overturn Ruling,NaN,NaN,CC,0015-10-28,Wizards88_Magic87,Q4 00:03.5 Instant Replay: Overturn Ruling nan...,00:03.5,"After communicating with the Replay Center, th...",NaN,Wizards,Magic
7247,Q4,00:02.9 Turnover: Traveling,Nikola Vucevic,NaN,NaN,0015-10-28,Wizards88_Magic87,Q4 00:02.9 Turnover: Traveling Nikola Vucevic ...,00:02.9,NaN,Traveling,Wizards,Magic


In [114]:
df_l2m['Foulee'].value_counts()

Isaiah Thomas               113
Russell Westbrook           102
James Harden                 97
Kemba Walker                 90
DeMar DeRozan                83
Damian Lillard               83
John Wall                    77
Andrew Wiggins               77
LeBron James                 75
Paul George                  73
Kawhi Leonard                70
Gordon Hayward               69
DeMarcus Cousins             67
Jimmy Butler                 65
Kyrie Irving                 63
Mike Conley                  63
Dwyane Wade                  61
Anthony Davis                55
CJ McCollum                  53
Paul Millsap                 52
Dennis Schroder              47
Bradley Beal                 47
Jeff Teague                  47
Carmelo Anthony              46
Nicolas Batum                45
Reggie Jackson               45
Eric Bledsoe                 44
Goran Dragic                 44
Devin Booker                 43
Karl-Anthony Towns           42
Giannis Antetokounmpo        42
Kevin Du

In [115]:
df_l2m['Fouler'].value_counts()

Marcin Gortat               85
Al Horford                  79
Steven Adams                72
Cody Zeller                 72
Serge Ibaka                 68
Hassan Whiteside            67
DeMarcus Cousins            66
James Harden                65
Brook Lopez                 65
Karl-Anthony Towns          65
LaMarcus Aldridge           61
Marcus Smart                60
Giannis Antetokounmpo       59
Paul George                 58
Paul Millsap                58
Anthony Davis               54
Tyson Chandler              54
Mason Plumlee               53
Dwight Howard               53
Marc Gasol                  53
DeAndre Jordan              52
Robin Lopez                 51
Tristan Thompson            50
Draymond Green              50
Carmelo Anthony             50
Myles Turner                50
Isaiah Thomas               48
Mike Conley                 47
Dwyane Wade                 47
LeBron James                46
Markieff Morris             46
Robert Covington            45
John Wal

In [116]:
df_l2m[['Home_team','Score']].groupby(['Home_team','Score']).count()

Empty DataFrame
Columns: []
Index: [(76ers, Bucks109_76ers108(OT)), (76ers, Celtics84_76ers80), (76ers, Grizzlies104_76ers99(2OT) ), (76ers, Heat103_76ers98), (76ers, Jazz95_76ers91), (76ers, Kings102_76ers100), (76ers, Knicks110_76ers109), (76ers, Knicks119_76ers113(2OT)), (76ers, Magic112_76ers109(OT)), (76ers, Nets100_76ers91), (76ers, Nuggets104_76ers103), (76ers, Pacers122_76ers115(OT)), (76ers, Thunder103_76ers97), (76ers, Timberwolves112_76ers111), (76ers, Wizards103_76ers94), (Bucks, 76ers114_Bucks109), (Bucks, Bulls102_Bucks98), (Bucks, Cavaliers114_Bucks108(OT)), (Bucks, Celtics112_Bucks109), (Bucks, Hornets87_Bucks82), (Bucks, Hornets98_Bucks95), (Bucks, Jazz84_Bucks81), (Bucks, Knicks116_Bucks111), (Bucks, Magic107_Bucks98), (Bucks, Mavericks86_Bucks79(OT)), (Bucks, Pistons105_Bucks108(OT)), (Bucks, Raptors106_Bucks100), (Bucks, Raptors92_Bucks89), (Bucks, Spurs97_Bucks96), (Bucks, Wizards110_Bucks105), (Bulls, Celtics107_Bulls100), (Bulls, Grizzlies98_Bulls91), (Bulls, Hawks102_Bulls100), (Bulls, Hawks104_Bulls106), (Bulls, Hawks119_Bulls114), (Bulls, Lakers96_Bulls90), (Bulls, Magic98_Bulls91), (Bulls, Nets107_Bulls106), (Bulls, Nuggets110_Bulls107 ), (Bulls, Nuggets115_Bulls110), (Bulls, Pacers111_Bulls101), (Bulls, Raptors118_Bulls123(OT)), (Bulls, Raptors122_Bulls120(OT)), (Bulls, Suns103_Bulls101), (Bulls, Timberwolves102_Bulls93(OT)), (Bulls, Timberwolves112_Bulls105), (Bulls, Timberwolves99_Bulls94), (Bulls, Warriors106_Bulls94), (Bulls, Wizards101_Bulls99), (Cavaliers, Celtics103_Cavaliers99), (Cavaliers, Celtics104_Cavaliers103), (Cavaliers, Celtics111_Cavaliers108), (Cavaliers, Hawks126_Cavaliers125(OT)), (Cavaliers, Heat124_Cavaliers121(OT)), (Cavaliers, Kings116_Cavaliers112(OT)), (Cavaliers, Pelicans114_Cavaliers108(OT)), (Cavaliers, Raptors105_Cavaliers99), (Cavaliers, Raptors99_Cavaliers97), (Cavaliers, Rockets106_Cavaliers100), (Cavaliers, Spurs118_Cavaliers115(OT)), (Cavaliers, Warriors118_Cavaliers113), (Celtics, 76ers105_Celtics99), (Celtics, 76ers106_Celtics110), (Celtics, Bucks103_Celtics100), (Celtics, Bulls105_Celtics99), (Celtics, Bulls106_Celtics102), (Celtics, Cavaliers124_Celtics118 ), (Celtics, Hawks102_Celtics101), (Celtics, Magic116_Celtics117), (Celtics, Pelicans106_Celtics105), (Celtics, Pistons119_Celtics116), (Celtics, Raptors114_Celtics106), (Celtics, Raptors115_Celtics109), (Celtics, Rockets107_Celtics106), (Celtics, Spurs108_Celtics105 ), (Celtics, Suns109_Celtics106), (Celtics, TrailBlazers116_Celtics109), (Celtics, TrailBlazers127_Celtics123(OT) ), (Celtics, Wizards119_Celtics129(OT)), (Celtics, Wizards92_Celtics91), (Clippers, Bucks97_Clippers96), (Clippers, Grizzlies111_Clippers107), (Clippers, Jazz105_Clippers98), (Clippers, Jazz96_Clippers92), (Clippers, Jazz97_Clippers95), (Clippers, Nuggets106_Clippers102), (Clippers, Nuggets87_Clippers81), (Clippers, Rockets109_Clippers105), (Clippers, Thunder85_Clippers83), (Clippers, Timberwolves104_Clippers101), (Clippers, TrailBlazers106_Clippers103), (Clippers, TrailBlazers96_Clippers88), (Clippers, Wizards117_Clippers110), (Grizzlies, Bulls108_Grizzlies104), (Grizzlies, Celtics112_Grizzlies109(OT)), (Grizzlies, Jazz92_Grizzlies87(OT)), (Grizzlies, Kings91_Grizzlies90), (Grizzlies, Kings96_Grizzlies92), (Grizzlies, Lakers108_Grizzlies103), (Grizzlies, Pelicans95_Grizzlies91), ...]

[472 rows x 0 columns]

In [126]:
df_l2m['Home_team'].value_counts()

TrailBlazers    368
Heat            340
Nets            331
Hornets         315
Celtics         311
Timberwolves    295
Kings           281
Bulls           277
Jazz            266
Pacers          260
Thunder         260
Suns            250
Mavericks       245
Nuggets         241
Knicks          236
76ers           233
Grizzlies       229
Rockets         227
Pelicans        226
Pistons         224
Bucks           224
Lakers          219
Cavaliers       209
Clippers        201
Hawks           194
Raptors         179
Spurs           168
Wizards         156
Warriors        135
Magic           134
LAClippers       14
Name: Home_team, dtype: int64

In [129]:
len(df_l2m['Home_team'].value_counts())

30

In [130]:
len(df_l2m['Away_team'].value_counts())

30

In [131]:
df_l2m['Away_team'].value_counts()

Wizards         414
Spurs           391
Rockets         352
Raptors         346
Celtics         331
Grizzlies       321
Cavaliers       314
Hawks           307
Thunder         290
Jazz            274
Heat            263
Hornets         247
TrailBlazers    238
Pelicans        237
Kings           228
Magic           223
Pistons         222
Timberwolves    217
Bulls           214
Knicks          209
Pacers          209
Bucks           209
76ers           203
Suns            193
Clippers        170
Warriors        153
Nuggets         152
Nets            110
Mavericks       106
Lakers          105
Name: Away_team, dtype: int64

In [68]:
df_l2m.replace(r'Sixers','76ers',inplace=True)

In [69]:
df_l2m.replace(r'Cleveland','Cavaliers',inplace=True)

In [70]:
df_l2m.replace(r'Dallas','Mavericks',inplace=True)

In [71]:
df_l2m.replace(r'Pelican','Pelicans',inplace=True)

df_l2m.replace(r'Grizzlies89_Pelican83(OT)','Grizzlies89_Pelicans83(OT)',inplace=True)

In [85]:
df_l2m.replace(r'Cleveland135_Atlanta130','Cavaliers135_Hawks130',inplace=True)
df_l2m.replace('Atlanta','Hawks',inplace=True)

In [84]:
df_l2m.replace(r'Bucks90_TrailBlazer88','Bucks90_TrailBlazers88',inplace=True)
df_l2m.replace('TrailBlazer','TrailBlazers',inplace=True)

In [82]:
df_l2m.replace(r'Magic96_Timberwoves93','Magic96_Timberwolves93',inplace=True)
df_l2m.replace('Timberwoves','Timberwolves',inplace=True)

In [94]:
df_l2m['Score'].unique()

array(['Warriors118_Cavaliers113', 'Celtics111_Cavaliers108',
       'Warriors113_Spurs111', 'Wizards92_Celtics91',
       'Rockets107_Spurs110(OT)', 'Wizards119_Celtics129(OT)',
       'Clippers98_Jazz93', 'Spurs103_Grizzlies96', 'Raptors92_Bucks89',
       'Wizards103_Hawks99', 'Jazz96_Clippers92',
       'Cavaliers106_Pacers102', 'Jazz105_Clippers98',
       'Grizzlies110_Spurs108(OT)', 'Thunder115_Rockets113',
       'Clippers11_Jazz106', 'Cavaliers119_Pacers114',
       'Wizards109_Hawks101', 'Rockets115_Thunder111',
       'Raptors106_Bucks100', 'Bulls106_Celtics102', 'Jazz97_Clippers95',
       'Magic113_Pistons109', 'Thunder100_Timberwolves98',
       'Heat124_Cavaliers121(OT)', 'TrailBlazers99_Spurs98',
       'Hawks126_Cavaliers125(OT)', 'Lakers110_Timberwolves109\xa0',
       'Nets107_Bulls106', 'Heat106_Wizards103', 'Celtics121_Hornets114',
       'Pistons114_Rockets109', 'Jazz120_Timberwolves113',
       'Lakers98_Kings94', 'Wizards106_Knicks103',
       'Rockets110_Nugget

In [103]:
df_l2m[df_l2m['Home_team']=='']

,Period,data,Fouler,Foulee,Review,Date,Score,Soup,Time,Comment,Foul,Away_team,Home_team
6568,Q4,01:33.0 Foul: Shooting,Jameer Nelson,Gordon Hayward,CC,0015-12-18,,Q4 01:33.0 Foul: Shooting Jameer Nelson Gordon...,01:33.0,Nelson (DEN) makes contact with Hayward's (UTA...,Shooting,,
6569,Q4,01:21.0 Turnover: Traveling,Will Barton,NaN,NaN,0015-12-18,,Q4 01:21.0 Turnover: Traveling Will Barton nan...,01:21.0,[Observable in enhanced video] Barton (DEN) ho...,Traveling,,
6570,Q4,00:56.1 Turnover: Offensive Goaltending,Gordon Hayward,NaN,CC,0015-12-18,,Q4 00:56.1 Turnover: Offensive Goaltending Gor...,00:56.1,Hayward (UTA) makes contact with the ball whil...,Offensive,,
6571,Q4,00:56.1 Instant Replay: Support Ruling,NaN,NaN,CC,0015-12-18,,Q4 00:56.1 Instant Replay: Support Ruling nan ...,00:56.1,"After communicating with the Replay Center, th...",NaN,,
6572,Q4,00:50.7 Foul: Offensive,Joffrey Lauvergne,Gordon Hayward,CNC,0015-12-18,,Q4 00:50.7 Foul: Offensive Joffrey Lauvergne G...,00:50.7,Lauvergne (DEN) sets the screen on Hayward (UT...,Offensive,,
6573,Q4,00:44.3 Turnover: 5 Second Inbound,",",NaN,CC,0015-12-18,,"Q4 00:44.3 Turnover: 5 Second Inbound , nan CC...",00:44.3,Barton (DEN) does not inbound the ball before ...,NaN,,
6574,Q4,00:28.7 Foul: Shooting,Gary Harris,Gordon Hayward,CNC,0015-12-18,,Q4 00:28.7 Foul: Shooting Gary Harris Gordon H...,00:28.7,Harris (DEN) maintains legal guarding position...,Shooting,,
6575,Q4,00:28.3 Turnover: Traveling,Gordon Hayward,NaN,INC,0015-12-18,,Q4 00:28.3 Turnover: Traveling Gordon Hayward ...,00:28.3,Hayward (UTA) gathers the ball with his right ...,Traveling,,
6576,Q4,00:11.8 Foul: Personal,Joffrey Lauvergne,Rodney Hood,CC,0015-12-18,,Q4 00:11.8 Foul: Personal Joffrey Lauvergne Ro...,00:11.8,NaN,Personal,,


In [102]:
df_l2m.replace(r'Philadelphia105_New York102','76ers105_Knicks102',inplace=True)
df_l2m.loc[1628:1647,'Score'] = '76ers105_Knicks102'
df_l2m.loc[1628:1647,'Away_team'] = '76ers'
df_l2m.loc[1628:1647,'Home_team'] = 'Knicks'

In [105]:
# last errors are for Jazz97_Nuggets88
index = df_l2m[df_l2m['Home_team']==''].index
df_l2m.loc[index,'Score'] = 'Jazz97_Nuggets88'
df_l2m.loc[index,'Away_team'] = 'Jazz'
df_l2m.loc[index,'Home_team'] = 'Nuggets'

In [107]:
df_l2m[df_l2m['Away_team']=='']

,Period,data,Fouler,Foulee,Review,Date,Score,Soup,Time,Comment,Foul,Away_team,Home_team


In [132]:
df_l2m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7248 entries, 0 to 7247
Data columns (total 13 columns):
Period       7248 non-null object
data         7248 non-null object
Fouler       6820 non-null object
Foulee       5952 non-null object
Review       6701 non-null object
Date         7248 non-null object
Score        7248 non-null object
Soup         7248 non-null object
Time         7247 non-null object
Comment      6775 non-null object
Foul         6272 non-null object
Away_team    7248 non-null object
Home_team    7248 non-null object
dtypes: object(13)
memory usage: 736.2+ KB


In [133]:
df_l2m.to_csv('./data/L2M_processed.csv', index=False, compression='gzip')

### Primary EDA

In [138]:
df_l2m[['Away_team','Score','Period']].groupby(['Away_team','Score'])['Period'].max()

Away_team     Score                          
76ers         76ers103_Nets98                    Q4
              76ers105_Celtics99                 Q4
              76ers105_Knicks102                 Q4
              76ers106_Celtics110                Q4
              76ers106_Nets101                   Q4
              76ers109_Wizards102                Q4
              76ers110_Kings105                  Q4
              76ers114_Bucks109                  Q4
              76ers118_Lakers116                 Q4
              76ers122_Kings119                  Q4
              76ers93_Timberwolves91             Q4
              76ers93_TrailBlazers92             Q4
              76ers94_Raptors89                  Q4
Bucks         Bucks102_Timberwolves95            Q4
              Bucks103_Celtics100                Q4
              Bucks103_Nets96                    Q4
              Bucks107_Magic100                  Q4
              Bucks109_76ers108(OT)              Q5
              Buck

In [8]:
periods_total = df_l2m[['Away_team','Score','Period']].groupby('Score')['Period','Away_team'].max()

In [9]:
periods_total.groupby('Away_team')['Period'].value_counts()

Away_team     Period
76ers         Q4        13
Bucks         Q4        14
              Q5         1
Bulls         Q4        14
              Q5         2
Cavaliers     Q4        18
              Q5         3
Celtics       Q4        19
              Q5         2
Clippers      Q4         8
              Q5         2
Grizzlies     Q4        17
              Q5         4
              Q6         1
Hawks         Q4        17
              Q5         3
Heat          Q4        17
              Q5         1
Hornets       Q4        15
              Q5         2
Jazz          Q4        13
              Q5         4
Kings         Q4        16
Knicks        Q4        10
              Q5         3
Lakers        Q4         7
Magic         Q4        13
              Q5         2
Mavericks     Q5         4
              Q4         2
Nets          Q4         7
              Q5         1
Nuggets       Q4        11
Pacers        Q4        10
              Q5         3
Pelicans      Q4        11
       

In [10]:
periods_total = periods_total.groupby('Away_team')['Period'].value_counts()
periods_total['Wizards']

Period
Q4    23
Q5     4
Name: Period, dtype: int64

In [11]:
df_teams = pd.DataFrame()

def plus(df, row_index, col_name, add_num):
    try:
        df.loc[row_index, col_name] += add_num
    except:
        df.loc[row_index, col_name] = add_num

for tupl in periods_total.items():
    (team, period), count = tupl
    period = int(period.strip('Q'))
    if period == 4: m = 2
    elif period >= 5: m = 5
    plus(df_teams, team, 'Total_l2m_minutes', count*m) # 2 min for each 4th quarter

In [12]:
df_teams

,Total_l2m_minutes
76ers,26.0
Bucks,33.0
Bulls,38.0
Cavaliers,51.0
Celtics,48.0
Clippers,26.0
Grizzlies,59.0
Hawks,49.0
Heat,39.0
Hornets,40.0


In [13]:
len(df_teams)

30

In [14]:
df_usage.head()

,Jabari Parker_AdjUsage,Raul Neto_AdjUsage,Khris Middleton_AdjUsage,Rodney Hood_AdjUsage,Rudy Gobert_AdjUsage,Michael Carter-Williams_AdjUsage,Michael Carter-Williams_<5ft,Gordon Hayward,UTA_Personal,MIL_Personal_given,Gordon Hayward_AdjUsage,Raul Neto,Greg Monroe_AdjUsage,Derrick Favors_AdjUsage,Raul Neto_<5ft,Khris Middleton_<5ft,Giannis Antetokounmpo_AdjUsage,Giannis Antetokounmpo_<5ft,Derrick Favors_<5ft,Jabari Parker_<5ft,Greg Monroe,MIL_Shooting,UTA_Shooting_given,Derrick Favors,UTA_Shooting,MIL_Shooting_given,Rudy Gobert,Chris Johnson_AdjUsage,Jerryd Bayless_AdjUsage,Johnny O'Bryant III_AdjUsage,Miles Plumlee_AdjUsage,Chris Johnson,Trey Burke_AdjUsage,Rodney Hood_<5ft,Tyler Ennis_AdjUsage,Trevor Booker_AdjUsage,Trevor Booker_<5ft,Tyler Ennis,MIL_Personal,UTA_Personal_given,Johnny O'Bryant III_<5ft,Trey Lyles_AdjUsage,Rashad Vaughn_AdjUsage,Rashad Vaughn_<5ft,Jerryd Bayless,...,Edy Tavares_5-10ft,Dahntay Jones,Sheldon McClellan_5-10ft,Jarrod Uthoff_5-10ft,Malik Beasley_5-10ft,Jamal Murray_10-15ft,Tim Quarterman_<5ft,Ron Baker_5-10ft,Chris McCullough_10-15ft,Michael Gbinije,John Wall_10-15ft,Richaun Holmes_10-15ft,Mike Dunleavy_5-10ft,Jameer Nelson_10-15ft,Ben McLemore_10-15ft,Stephen Curry_15-22ft,Justin Hamilton_5-10ft,Shaun Livingston_15-22ft,John Lucas III_AdjUsage,John Lucas III_<5ft,Andrew Nicholson_5-10ft,Demetrius Jackson_<5ft,Demetrius Jackson,Tyus Jones_15-22ft,Thabo Sefolosha_5-10ft,Nicolas Laprovittola_5-10ft,AJ Hammons_<5ft,Robert Covington_10-15ft,Danuel House_AdjUsage,DeAndre' Bembry_10-15ft,Randy Foye_10-15ft,Pat Connaughton_5-10ft,Kevin Durant_23+ft,DeAndre Liggins_5-10ft,Jonathan Gibson_10-15ft,Vince Carter_10-15ft,Ryan Kelly_5-10ft,Mike Muscala_10-15ft,Tyler Johnson_15-22ft,Joe Harris_5-10ft,Myles Turner_10-15ft,E'Twaun Moore_15-22ft,Buddy Hield_15-22ft,Tim Frazier_15-22ft,Score_label
21500758,14.0,18.0,40.0,25.0,23.0,37.0,7.0,2.0,7.0,7.0,26.0,2.0,25.0,38.0,2.0,2.0,25.0,4.0,4.0,2.0,3.0,9.0,9.0,4.0,7.0,7.0,3.0,9.0,15.0,11.0,9.0,2.0,4.0,2.0,4.0,7.0,2.0,2.0,7.0,7.0,1.0,10.0,2.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bucks84_Jazz81
21500991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bulls107_Raptors109
21500538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jazz103_Rockets94
21500947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hawks84_Jazz91
21500095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lakers99_Knicks95


In [15]:
new_index = df.groupby('GAME_ID')['Score_label'].max()
len(new_index)

2460

In [16]:
new_index

GAME_ID
21500001                 Pistons94_Hawks106
21500002                Cavaliers97_Bulls95
21500003             Pelicans111_Warriors95
21500004                  Wizards87_Magic88
21500005                 76ers112_Celtics95
21500006                   Bulls100_Nets115
21500007                   Jazz92_Pistons87
21500008                  Hornets104_Heat94
21500009                Pacers106_Raptors99
21500010               Nuggets85_Rockets105
21500011           Cavaliers76_Grizzlies106
21500012                  Knicks97_Bucks122
21500013                Spurs112_Thunder106
21500014                Mavericks95_Suns111
21500015         Pelicans112_TrailBlazers94
21500016               Clippers104_Kings111
21500017          Timberwolves111_Lakers112
21500018             Grizzlies103_Pacers112
21500019                 Hawks101_Knicks112
21500020            Mavericks104_Clippers88
21500021                Heat102_Cavaliers92
21500022                Thunder136_Magic139
21500023                

In [17]:
df_usage['Score_label'] = new_index

In [18]:
df_usage.filter(regex='WAS.*').sum()

WAS_Personal             9.0
WAS_Personal_given       8.0
WAS_Offensive            1.0
WAS_Shooting_given       7.0
WAS_Shooting            13.0
WAS_Loose Ball           2.0
WAS_Traveling            1.0
WAS_Loose Ball_given     1.0
WAS_Offensive_taken      1.0
dtype: float64

In [19]:
df_usage.filter(regex='SAS.*').sum()

SAS_Personal             7.0
SAS_Offensive_taken      3.0
SAS_Shooting             8.0
SAS_Personal_given      13.0
SAS_Loose Ball           4.0
SAS_Traveling            2.0
SAS_Shooting_given       7.0
SAS_Loose Ball_given     2.0
SAS_Offensive            1.0
dtype: float64

In [20]:
df_usage.filter(regex='POR.*').sum()

POR_Personal_given      13.0
POR_Shooting             6.0
POR_Personal             8.0
POR_Offensive            1.0
POR_Shooting_given       8.0
POR_Loose Ball           2.0
POR_Loose Ball_given     2.0
POR_Offensive_taken      1.0
POR_Traveling            1.0
dtype: float64

In [21]:
df_usage.filter(regex='James Harden.*').sum()

James Harden_AdjUsage    1312.0
James Harden_<5ft          11.0
James Harden               12.0
James Harden_10-15ft        1.0
James Harden_5-10ft         4.0
dtype: float64

### Foul Counts in df_usage are off, Re-making All Foul Columns

In [190]:
df['Foul'].value_counts()

S.FOUL                  46028
P.FOUL                  31541
L.B.FOUL                 6058
OFF.Foul                 5445
Personal                 4390
Traveling                3970
Turnover: Shot Clock     2637
Shooting                 2176
T.Foul (Def. 3 Sec        212
Name: Foul, dtype: int64

In [191]:
df['Fouler'].value_counts()

Giannis Antetokounmpo     523
Gorgui Dieng              513
DeMarcus Cousins          503
Julius Randle             492
Andre Drummond            490
Alex Len                  489
Karl-Anthony Towns        485
Mason Plumlee             470
Draymond Green            464
Patrick Beverley          463
Kristaps Porzingis        462
Hassan Whiteside          460
Devin Booker              456
JaMychal Green            448
Robert Covington          447
Bismack Biyombo           441
Serge Ibaka               439
Markieff Morris           438
Nikola Jokic              437
Dwight Howard             430
Paul Millsap              428
James Harden              425
Amir Johnson              423
Marcin Gortat             423
DeAndre Jordan            420
Rudy Gobert               419
Brook Lopez               419
Paul George               419
Myles Turner              418
Matt Barnes               411
Ricky Rubio               410
Steven Adams              408
PJ Tucker                 406
Evan Fourn

In [192]:
test = pd.DataFrame(data=[[1,2],[3,4]])
test

,0,1
0,1,2
1,3,4


In [129]:
usage_types1 = ['Layup', 'Dunk', 'Driving'] # Common situations to foul, x2 weight
usage_types2 = ['Shot', 'shot', 'Jumper', 'Turnover', 'REBOUND']
usage_type3 = 'AST' # only type which co-occurs during another event type

def plus1(df, row_index, col_name):
    try:
        if np.isnan(df.loc[row_index, col_name]):
            df.loc[row_index, col_name] = 1
        else:
            df.loc[row_index, col_name] += 1
    except:
        df.loc[row_index, col_name] = 1

def make_usage(df, df_usage):
    
    c = 0
    for idx, row in df.iterrows():
        c += 1
        if c % 50000 == 0: print(c, ' rows processed')

        if type(row['Foul']) == float:
        
            usage1done = False
            usage2done = False
            if type(row['VISITORDESCRIPTION']) == str: 
                text = row['VISITORDESCRIPTION']
            elif (type(row['HOMEDESCRIPTION']) == str):
                text = row['HOMEDESCRIPTION']
            else:
                text = ''
                usage1done = True
                usage2done = True                
# Not using home versus away splits, because there is too low a total count of fouls
# Biggest hypothesized factor in foul bias would be strength of opponent, and can't quantify
# that accurately (ex: strength of schedule) given labeled data restricted to last 2 minutes
#                 is_home_text = 'error'
#                 player2text = 'error'
            
            for usage in usage_types1:
                if (usage in text) & (not usage1done):
                    usage1done = True
                    plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_AdjUsage')
                    # Plus 1 twice due to double weight for driving shots
                    plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_AdjUsage')
                    match = re.search("(\d+)\'", text)
                    try:
                        shot_dist = int(match.group(1))
                        if shot_dist <= 5:
                            plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_<5ft')
                        elif shot_dist <= 10:
                            plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_5-10ft')
                        elif shot_dist <= 15:
                            plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_10-15ft')
                        elif shot_dist <= 22:
                            plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_15-22ft')
                        else:
                            plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_23+ft')
                    except: pass

            if 'AST' in text:
                plus1(df_usage, row['GAME_ID'], str(row['PLAYER2_NAME']) + '_AdjUsage')
                
            if not usage1done:
                for usage in usage_types2:
                    if (usage in text) & (not usage2done):
                        usage2done = True
                        plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_AdjUsage')

        elif type(row['Foul']) == str:
            if row['Foul'] in foul_types_dict.keys():
                foul = foul_types_dict[row['Foul']]
                if foul in ['Personal', 'Shooting', 'Loose Ball']:
                    plus1(df_usage, row['GAME_ID'], str(row['Foulee']) + '_' + foul)
                    plus1(df_usage, row['GAME_ID'], str(row['Foulee_team']) + '_' + foul)
                    plus1(df_usage, row['GAME_ID'], str(row['Fouler_team']) + '_'+foul+'_given')
                    plus1(df_usage, row['GAME_ID'], str(row['Foulee']) + '_AdjUsage')
                elif foul in ['Offensive', 'Traveling']:
                    plus1(df_usage, row['GAME_ID'], str(row['Fouler']) + '_' + foul)
                    plus1(df_usage, row['GAME_ID'], str(row['Fouler_team']) + '_' + foul)
                    plus1(df_usage, row['GAME_ID'], str(row['Foulee_team']) + '_'+foul+'_taken')
                    plus1(df_usage, row['GAME_ID'], str(row['Fouler']) + '_AdjUsage')
                elif foul == '24 Second':
                    plus1(df_usage, row['GAME_ID'], str(row['Fouler_team']) + '_24_Sec')


In [130]:
df_usage = pd.DataFrame()

make_usage(df, df_usage)

50000  rows processed
100000  rows processed
150000  rows processed
200000  rows processed
250000  rows processed
300000  rows processed
350000  rows processed
400000  rows processed
450000  rows processed
500000  rows processed
550000  rows processed
600000  rows processed
650000  rows processed
700000  rows processed
750000  rows processed
800000  rows processed
850000  rows processed
900000  rows processed
950000  rows processed
1000000  rows processed
1050000  rows processed
1100000  rows processed


In [128]:
match = re.search("(\d+)\'", "Neto 14' Fadeaway Jumper (2 PTS)")
shot_dist = int(match.group(1))
shot_dist

14

In [131]:
df_usage.head()

,Jabari Parker_AdjUsage,Raul Neto_AdjUsage,Khris Middleton_AdjUsage,Rodney Hood_AdjUsage,Rudy Gobert_AdjUsage,Michael Carter-Williams_AdjUsage,Michael Carter-Williams_<5ft,Gordon Hayward_Personal,UTA_Personal,MIL_Personal_given,Gordon Hayward_AdjUsage,Raul Neto_Personal,Greg Monroe_AdjUsage,Derrick Favors_AdjUsage,Raul Neto_<5ft,Khris Middleton_<5ft,Giannis Antetokounmpo_AdjUsage,Giannis Antetokounmpo_<5ft,Derrick Favors_<5ft,Jabari Parker_<5ft,Greg Monroe_Shooting,MIL_Shooting,UTA_Shooting_given,Derrick Favors_Shooting,UTA_Shooting,MIL_Shooting_given,Rudy Gobert_Shooting,Chris Johnson_AdjUsage,Jerryd Bayless_AdjUsage,Johnny O'Bryant III_AdjUsage,Miles Plumlee_AdjUsage,Chris Johnson_Personal,Trey Burke_AdjUsage,Rodney Hood_<5ft,Tyler Ennis_AdjUsage,Trevor Booker_AdjUsage,Trevor Booker_<5ft,Chris Johnson_Shooting,Tyler Ennis_Personal,MIL_Personal,UTA_Personal_given,Johnny O'Bryant III_<5ft,Trey Lyles_AdjUsage,Rashad Vaughn_AdjUsage,Rashad Vaughn_<5ft,...,Stephen Curry_15-22ft,Justin Holiday_Offensive,Montrezl Harrell_Traveling,Kyle Wiltjer_Personal,Anthony Morrow_Traveling,Justin Hamilton_5-10ft,Shaun Livingston_15-22ft,John Lucas III_AdjUsage,John Lucas III_<5ft,Andrew Nicholson_5-10ft,Denzel Valentine_Loose Ball,Demetrius Jackson_<5ft,Demetrius Jackson_Personal,Demetrius Jackson_Shooting,Demetrius Jackson_Loose Ball,Jarnell Stokes_Loose Ball,Dragan Bender_Traveling,J.J. Barea_Loose Ball,Tyus Jones_15-22ft,Thabo Sefolosha_5-10ft,Nicolas Laprovittola_5-10ft,AJ Hammons_<5ft,Malik Beasley_Loose Ball,Sergio Rodriguez_Offensive,Robert Covington_10-15ft,Danuel House_AdjUsage,DeAndre' Bembry_10-15ft,Randy Foye_10-15ft,Pat Connaughton_5-10ft,Kay Felder_Loose Ball,Kevin Durant_23+ft,Tomas Satoransky_Offensive,DeAndre Liggins_5-10ft,Georges Niang_Loose Ball,Jonathan Gibson_10-15ft,Jonathan Gibson_Loose Ball,Vince Carter_10-15ft,Ryan Kelly_5-10ft,Mike Muscala_10-15ft,Tyler Johnson_15-22ft,Joe Harris_5-10ft,Myles Turner_10-15ft,E'Twaun Moore_15-22ft,Buddy Hield_15-22ft,Tim Frazier_15-22ft
21500758,14.0,18.0,40.0,25.0,23.0,37.0,7.0,1.0,7.0,7.0,26.0,2.0,25.0,38.0,2.0,2.0,25.0,4.0,4.0,2.0,2.0,9.0,9.0,2.0,7.0,7.0,2.0,9.0,15.0,11.0,9.0,1.0,4.0,2.0,4.0,7.0,2.0,1.0,2.0,7.0,7.0,1.0,10.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21500991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21500538,NaN,19.0,NaN,16.0,10.0,NaN,NaN,1.0,5.0,NaN,29.0,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,11.0,NaN,NaN,12.0,NaN,NaN,NaN,NaN,24.0,NaN,NaN,26.0,3.0,NaN,NaN,NaN,6.0,NaN,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21500947,NaN,12.0,NaN,29.0,31.0,NaN,NaN,3.0,5.0,NaN,38.0,NaN,NaN,37.0,2.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,6.0,3.0,13.0,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,14.0,3.0,NaN,NaN,NaN,5.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21500095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
df_usage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2460 entries, 21500758 to 21600130
Columns: 4794 entries, Jabari Parker_AdjUsage to Tim Frazier_15-22ft
dtypes: float64(4794)
memory usage: 90.1 MB


In [132]:
df_usage.filter(regex='POR.*').sum()

POR_Personal_given      1137.0
POR_Shooting            1613.0
POR_Personal            1161.0
POR_Offensive            191.0
POR_Shooting_given      1856.0
POR_Loose Ball           176.0
POR_Loose Ball_given     215.0
POR_Offensive_taken      191.0
POR_Traveling            125.0
dtype: float64

### Analyzing by Player

In [ ]:
# Not enough Loose Ball, Traveling and 24 second violations to analyze in any possible subgroupings
# For Personal fouls, won't analyze at player level because many close game personal fouls are
# intentional, when their team needs the ball back and fouls immediately
# Will focus on Shooting and Offensive Fouls

In [50]:
df_l2m['Foul'].value_counts()

Personal      2070
Shooting      1838
Offensive     1351
Loose Ball     584
Traveling      382
24 Second       47
Name: Foul, dtype: int64

In [65]:
df_l2m_player = pd.read_csv('./data/L2M_player.csv')
df_l2m_player.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3175 entries, 0 to 3174
Data columns (total 14 columns):
Period         3175 non-null object
data           3175 non-null object
Fouler_team    3175 non-null object
Fouler         3175 non-null object
Foulee         3174 non-null object
Review         3151 non-null object
Date           3175 non-null object
Score          3175 non-null object
Soup           3175 non-null object
Time           3175 non-null object
Comment        3022 non-null object
Foul           3175 non-null object
Away_team      3175 non-null object
Home_team      3175 non-null object
dtypes: object(14)
memory usage: 347.3+ KB


In [66]:
df_l2m_player = df_l2m_player[(df_l2m_player['Foul']=='Shooting') | (df_l2m_player['Foul']=='Offensive')]

In [67]:
df_l2m = None
df_l2m_player.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3175 entries, 0 to 3174
Data columns (total 14 columns):
Period         3175 non-null object
data           3175 non-null object
Fouler_team    3175 non-null object
Fouler         3175 non-null object
Foulee         3174 non-null object
Review         3151 non-null object
Date           3175 non-null object
Score          3175 non-null object
Soup           3175 non-null object
Time           3175 non-null object
Comment        3022 non-null object
Foul           3175 non-null object
Away_team      3175 non-null object
Home_team      3175 non-null object
dtypes: object(14)
memory usage: 372.1+ KB


In [115]:
def plus1(df, row_index, col_name):
    try:
        if np.isnan(df.loc[row_index, col_name]):
            df.loc[row_index, col_name] = 1
        else:
            df.loc[row_index, col_name] += 1
    except:
        df.loc[row_index, col_name] = 1

def make_usage_player(df, df_usage_player):
    
    for idx, row in df.iterrows():

        away_team = row['Away_team']
        Foulee_team = (row['Home_team'] if away_team == row['Fouler_team'] else away_team)
        if row['Foul']=='Shooting':
            plus1(df_usage_player, row['Score'], str(row['Foulee']) + '_Shooting_' + str(row['Review']))
            plus1(df_usage_player, row['Score'], str(Foulee_team) + '_Shooting_' + str(row['Review']))
            plus1(df_usage_player, row['Score'], str(row['Fouler_team']) + '_Shooting_given_' + str(row['Review']))

        elif row['Foul']=='Offensive':
            plus1(df_usage_player, row['Score'], str(row['Fouler']) + '_Offensive_' + str(row['Review']))
            plus1(df_usage_player, row['Score'], str(row['Fouler_team']) + '_Offensive_' + str(row['Review']))
            plus1(df_usage_player, row['Score'], str(Foulee_team) + '_Offensive_taken_' + str(row['Review']))
        if (str(row['Foulee']) == 'James Harden'):# | (str(row['Fouler']) == 'James Harden'):
            print(row['Score'], str(row['Foulee']) + '_Shooting', row['Review'],
                  df_usage_player.loc[row['Score'], str(row['Foulee']) + '_Shooting_' + str(row['Review'])])

In [116]:
df_usage_l2m = pd.DataFrame()

make_usage_player(df_l2m_player, df_usage_l2m)

Rockets107_Spurs110(OT) James Harden_Shooting CNC 1.0
Thunder115_Rockets113 James Harden_Shooting CC 1.0
Thunder115_Rockets113 James Harden_Shooting CNC 1.0
Rockets115_Thunder111 James Harden_Shooting CNC 1.0
Pistons114_Rockets109 James Harden_Shooting CNC 1.0
Pistons114_Rockets109 James Harden_Shooting CNC 2.0
Pistons114_Rockets109 James Harden_Shooting CNC 3.0
Rockets110_Nuggets104  James Harden_Shooting CNC 1.0
TrailBlazers117_Rockets107 James Harden_Shooting CNC 1.0
TrailBlazers117_Rockets107 James Harden_Shooting CNC 2.0
Rockets125_Nuggets124  James Harden_Shooting CNC 1.0
Jazz115_Rockets108 James Harden_Shooting CC 1.0
Jazz115_Rockets108 James Harden_Shooting CC 2.0
Spurs112_Rockets110 James Harden_Shooting CNC 1.0
Spurs112_Rockets110 James Harden_Shooting CC 1.0
Spurs112_Rockets110 James Harden_Shooting CNC 2.0
Bulls117_Rockets121(OT) James Harden_Shooting CC 1.0
Bulls117_Rockets121(OT) James Harden_Shooting CC 2.0
Hawks113_Rockets108 James Harden_Shooting INC 1.0
Hawks113_Rocke

In [105]:
df_usage_l2m.head()

,Kyrie Irving_Shooting,Cavaliers_Shooting,Warriors_Shooting_given,Kevin Love_Offensive,Cavaliers_Offensive,Warriors_Offensive_taken,Kyle Korver_Shooting,Kevin Durant_Shooting,Warriors_Shooting,Cavaliers_Shooting_given,LeBron James_Shooting,Kevin Love_Shooting,Celtics_Shooting_given,Al Horford_Offensive,Celtics_Offensive,Cavaliers_Offensive_taken,Marcus Smart_Shooting,Celtics_Shooting,Marcus Smart_Offensive,LaMarcus Aldridge_Offensive,Spurs_Offensive,Draymond Green_Offensive,Warriors_Offensive,Spurs_Offensive_taken,Shaun Livingston_Offensive,Draymond Green_Shooting,Spurs_Shooting_given,LaMarcus Aldridge_Shooting,Spurs_Shooting,Marcin Gortat_Offensive,Wizards_Offensive,Celtics_Offensive_taken,John Wall_Offensive,Isaiah Thomas_Shooting,Wizards_Shooting_given,John Wall_Shooting,Wizards_Shooting,Wizards_Offensive_taken,Bradley Beal_Shooting,Al Horford_Shooting,Trevor Ariza_Offensive,Rockets_Offensive,Rockets_Offensive_taken,Rockets_Shooting_given,James Harden_Shooting,...,Wayne Ellington_Offensive,Shane Larkin_Shooting,Chris Bosh_Offensive,Luol Deng_Offensive,Spencer Hawes_Offensive,Derrick Williams_Shooting,Hollis Thompson_Shooting,Isaiah Canaan_Shooting,Isaiah Canaan_Offensive,Lance Thomas_Shooting,Lou Amundson_Offensive,Mario Chalmers_Shooting,CJ Miles_Shooting,Kobe Bryant_Shooting,Raymond Felton_Shooting,Tibor Pleiss_Shooting,Omri Casspi_Offensive,Ty Lawson_Offensive,Langston Galloway_Shooting,Kobe Bryant_Offensive,Mike Miller_Offensive,Joffrey Lauvergne_Offensive,Jonas Jerebko_Offensive,Jonas Jerebko_Shooting,Andrea Bargnani_Shooting,Hollis Thompson_Offensive,Andrea Bargnani_Offensive,Jerami Grant_Offensive,Lavoy Allen_Shooting,Corey Brewer_Offensive,Alex Len_Shooting,Ramon Sessions_Shooting,Jrue Holiday_Offensive,Jarrett Jack_Shooting,Tony Snell_Offensive,Zaza Pachulia_Offensive,Metta World Peace_Offensive,Zaza Pachulia_Shooting,Brandon Bass_Shooting,Lavoy Allen_Offensive,Jordan Hill_Offensive,Tayshaun Prince_Shooting,Dewayne Dedmon_Offensive,Ty Lawson_Shooting,Mario Hezonja_Shooting
Warriors118_Cavaliers113,2.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Celtics111_Cavaliers108,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,3.0,4.0,4.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Warriors113_Spurs111,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wizards92_Celtics91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,5.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rockets107_Spurs110(OT),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,2.0,NaN,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,5.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
df_usage_l2m.filter(regex='James Harden.*').sum() 
# Should be very close to 55 shooting fouls for Harden 

James Harden_Shooting_CNC     31.0
James Harden_Offensive_CC      4.0
James Harden_Shooting_CC      17.0
James Harden_Offensive_CNC     8.0
James Harden_Shooting_INC      2.0
James Harden_Offensive_INC     1.0
dtype: float64

In [107]:
test = df_usage_l2m.filter(regex='James Harden.*').dropna(how='all')
df_usage_l2m.filter(regex='James Harden.*').dropna(how='all')

,James Harden_Shooting,James Harden_Offensive
Rockets107_Spurs110(OT),1.0,1.0
Thunder115_Rockets113,2.0,NaN
Rockets115_Thunder111,1.0,NaN
Pistons114_Rockets109,3.0,NaN
Rockets110_Nuggets104,1.0,1.0
TrailBlazers117_Rockets107,2.0,NaN
Rockets125_Nuggets124,1.0,NaN
Jazz115_Rockets108,2.0,NaN
Spurs112_Rockets110,3.0,NaN
Bulls117_Rockets121(OT),2.0,1.0


In [126]:
df_usage_l2m.filter(regex='POR.*').sum()

Thunder128_Knicks122(OT)           16
Pelicans125_Hornets122(OT)|        16
Suns107_Knicks105                  15
Rockets119_Magic114(OT             15
Suns118_TrailBlazers115(OT)        15
Hornets127_Kings122(OT)            14
Spurs88_Jazz86                     14
Timberwolves132_Wizards129(2OT)    14
Hornets100_Hawks96                 14
Magic112_Heat109                   14
Warriors121_Thunder118(OT)         14
Knicks107_TrailBlazers103          13
Spurs100_Thunder96                 13
Clippers107_Hornets102             13
Grizzlies110_Pelicans108(2OT)      13
Celtics105_Heat95                  13
Pacers115_Thunder111(OT)           13
Timberwolves102_Bulls93(OT)        12
Wizards125_TrailBlazers124(OT)     12
Pistons114_Hornets108(OT)          12
Raptors95_Heat91                   12
Cavaliers135_Hawks130              12
Thunder132_Nuggets129(OT)          12
Pelicans110_Suns108                12
Nets107_Bulls106                   12
Lakers108_Grizzlies103             12
Pelicans121_

In [127]:
df_usage_l2m.filter(regex='Rockets.*').sum()

Rockets_Offensive_INC          10.0
Rockets_Offensive_taken_CNC    42.0
Rockets_Shooting_given_CNC     43.0
Rockets_Shooting_CNC           43.0
Rockets_Offensive_CC            6.0
Rockets_Shooting_given_CC      19.0
Rockets_Offensive_CNC          47.0
Rockets_Shooting_CC            22.0
Rockets_Offensive_taken_INC     3.0
Rockets_Shooting_INC            6.0
Rockets_Offensive_taken_CC      5.0
Rockets_Shooting_given_INC      5.0
Rockets_Shooting_given_nan      1.0
Rockets_Offensive_taken_nan     2.0
dtype: float64

In [222]:
# Old incorrect totals
df_usage.filter(regex='James Harden.*').sum()

James Harden_AdjUsage    1312.0
James Harden_<5ft          11.0
James Harden               12.0
James Harden_10-15ft        1.0
James Harden_5-10ft         4.0
dtype: float64

In [138]:
df_usage.filter(regex='Russell Westbrook.*').sum()

Russell Westbrook_AdjUsage      9074.0
Russell Westbrook_<5ft           916.0
Russell Westbrook_Shooting       602.0
Russell Westbrook_Offensive       21.0
Russell Westbrook_Personal       327.0
Russell Westbrook_Traveling       13.0
Russell Westbrook_5-10ft          20.0
Russell Westbrook_Loose Ball      12.0
Russell Westbrook_15-22ft          1.0
Russell Westbrook_10-15ft          2.0
dtype: float64

In [137]:
df_usage.filter(regex='OKC.*').sum()

OKC_Shooting            1790.0
OKC_Shooting_given      1716.0
OKC_Loose Ball           280.0
OKC_Offensive            195.0
OKC_Personal_given      1142.0
OKC_Personal            1046.0
OKC_Traveling            144.0
OKC_Offensive_taken      147.0
OKC_Loose Ball_given     233.0
dtype: float64

In [139]:
df_usage_l2m.filter(regex='Russell Westbrook.*').sum() 

Russell Westbrook_Shooting_CC      15.0
Russell Westbrook_Shooting_CNC     35.0
Russell Westbrook_Offensive_CNC     5.0
Russell Westbrook_Shooting_INC      4.0
Russell Westbrook_Offensive_INC     1.0
dtype: float64

In [140]:
df_usage_l2m.filter(regex='James Harden.*').sum() 

James Harden_Shooting_CNC     31.0
James Harden_Offensive_CC      4.0
James Harden_Shooting_CC      17.0
James Harden_Offensive_CNC     8.0
James Harden_Shooting_INC      2.0
James Harden_Offensive_INC     1.0
dtype: float64

In [134]:
df_usage.filter(regex='James Harden.*').sum()

James Harden_AdjUsage      8248.0
James Harden_<5ft           754.0
James Harden_Shooting       649.0
James Harden_Personal       337.0
James Harden_Offensive       37.0
James Harden_Traveling       21.0
James Harden_Loose Ball      25.0
James Harden_10-15ft         16.0
James Harden_5-10ft          61.0
dtype: float64

In [136]:
df_usage.filter(regex='HOU.*').sum()

HOU_Shooting_given      1520.0
HOU_Shooting            1886.0
HOU_Offensive_taken      188.0
HOU_Personal            1081.0
HOU_Personal_given      1376.0
HOU_Offensive            152.0
HOU_Loose Ball           191.0
HOU_Traveling            140.0
HOU_Loose Ball_given     234.0
dtype: float64

In [23]:
# Old totals
df_usage.filter(regex='.*_AdjUsage').sum().sort_values(ascending=False).head(30)

Andre Drummond_AdjUsage           1451.0
James Harden_AdjUsage             1312.0
Brook Lopez_AdjUsage              1285.0
Kevin Durant_AdjUsage             1276.0
Russell Westbrook_AdjUsage        1219.0
Andrew Wiggins_AdjUsage           1170.0
Anthony Davis_AdjUsage            1169.0
Chris Paul_AdjUsage               1008.0
DeMarcus Cousins_AdjUsage         1004.0
Karl-Anthony Towns_AdjUsage        881.0
DeMar DeRozan_AdjUsage             866.0
Marc Gasol_AdjUsage                864.0
Jeff Teague_AdjUsage               785.0
Stephen Curry_AdjUsage             774.0
Marcin Gortat_AdjUsage             773.0
LaMarcus Aldridge_AdjUsage         765.0
Giannis Antetokounmpo_AdjUsage     765.0
DeAndre Jordan_AdjUsage            758.0
Carmelo Anthony_AdjUsage           716.0
Hassan Whiteside_AdjUsage          712.0
Jimmy Butler_AdjUsage              677.0
Marcus Morris_AdjUsage             666.0
Jrue Holiday_AdjUsage              653.0
Kyle Lowry_AdjUsage                652.0
Derrick Rose_Adj

In [121]:
df_usage.filter(regex='.*_AdjUsage').sum().sort_values(ascending=False).head(30)

Russell Westbrook_AdjUsage        9074.0
James Harden_AdjUsage             8248.0
LeBron James_AdjUsage             7537.0
DeMarcus Cousins_AdjUsage         7347.0
John Wall_AdjUsage                7044.0
Karl-Anthony Towns_AdjUsage       6691.0
Giannis Antetokounmpo_AdjUsage    6594.0
Stephen Curry_AdjUsage            6527.0
Damian Lillard_AdjUsage           6425.0
Andre Drummond_AdjUsage           6420.0
Isaiah Thomas_AdjUsage            6325.0
Anthony Davis_AdjUsage            6315.0
DeMar DeRozan_AdjUsage            6072.0
Paul George_AdjUsage              5869.0
Kemba Walker_AdjUsage             5760.0
Andrew Wiggins_AdjUsage           5645.0
Kevin Durant_AdjUsage             5576.0
Jimmy Butler_AdjUsage             5533.0
Draymond Green_AdjUsage           5413.0
Carmelo Anthony_AdjUsage          5412.0
DeAndre Jordan_AdjUsage           5354.0
Paul Millsap_AdjUsage             5348.0
CJ McCollum_AdjUsage              5310.0
Hassan Whiteside_AdjUsage         5226.0
Brook Lopez_AdjU

In [109]:
df_usage.to_csv('./data/usage.csv', index=True, compression='gzip')

In [25]:
df_teams.index

Index(['76ers', 'Bucks', 'Bulls', 'Cavaliers', 'Celtics', 'Clippers',
       'Grizzlies', 'Hawks', 'Heat', 'Hornets', 'Jazz', 'Kings', 'Knicks',
       'Lakers', 'Magic', 'Mavericks', 'Nets', 'Nuggets', 'Pacers', 'Pelicans',
       'Pistons', 'Raptors', 'Rockets', 'Spurs', 'Suns', 'Thunder',
       'Timberwolves', 'TrailBlazers', 'Warriors', 'Wizards'],
      dtype='object')

### Analyzing Top 10 Usage Players

In [143]:
# Using top 20 usage players in L2M that didn't switch teams
top10 = df_usage.filter(regex='.*_AdjUsage').sum().sort_values(ascending=False).head(11)

In [377]:
# Will only analyze players with highest minutes totals and adjusted usage over the whole season
# and played all for one team for the 15-16, 16-17 seasons

top10_l2m_minutes = pd.Series(
    data=[
        df_teams.loc['Thunder'], # Russell Westbrook_AdjUsage        9074.0
        df_teams.loc['Rockets'], # James Harden_AdjUsage             8248.0
        df_teams.loc['Cavaliers'], # LeBron James_AdjUsage             7537.0
        1, # DeMarcus Cousins_AdjUsage         7347.0
        df_teams.loc['Wizards'], # John Wall_AdjUsage                7044.0
        df_teams.loc['Timberwolves'], # Karl-Anthony Towns_AdjUsage       6691.0
        df_teams.loc['Bucks'], # Giannis Antetokounmpo_AdjUsage    6594.0
        df_teams.loc['Warriors'], # Stephen Curry_AdjUsage            6527.0
        df_teams.loc['TrailBlazers'], # Damian Lillard_AdjUsage           6425.0
        df_teams.loc['Pistons'], # Andre Drummond_AdjUsage           6420.0
        df_teams.loc['Celtics'] # Isaiah Thomas_AdjUsage            6325.0      
#         df_teams.loc['Pelicans'], # Anthony Davis_AdjUsage            6315.0
#         df_teams.loc['Raptors'], # DeMar DeRozan_AdjUsage            6072.0
#         df_teams.loc['Nets'], # Paul George_AdjUsage              5869.0
#         df_teams.loc['Hornets'], # Kemba Walker_AdjUsage             5760.0
#         df_teams.loc['Timberwolves'], # Andrew Wiggins_AdjUsage           5645.0
#         10000, # Excluded, switched teams, Kevin Durant_AdjUsage             5576.0
#         df_teams.loc['Bulls'], # Jimmy Butler_AdjUsage             5533.0
#         df_teams.loc['Warriors'], # Draymond Green_AdjUsage           5413.0
#         df_teams.loc['Knicks'], # Carmelo Anthony_AdjUsage          5412.0
#         df_teams.loc['Clippers'], # DeAndre Jordan_AdjUsage           5354.0
#         df_teams.loc['Hawks'], # Paul Millsap_AdjUsage             5348.0
         ], index=top10.index)

top10_season_minutes = pd.Series(data=[
        80*34.4 + 81*34.6, # Russell Westbrook_AdjUsage        9074.0
        82*38.1 + 81*36.4, # James Harden_AdjUsage             8248.0
        76*35.6 + 74*37.8, # LeBron James_AdjUsage             7537.0
        1, # DeMarcus Cousins_AdjUsage         7347.0
        77*36.2 + 78*36.4, # John Wall_AdjUsage                7044.0
        82*32.0 + 82*37.0, # Karl-Anthony Towns_AdjUsage       6691.0
        79*35.3 + 80*35.6, # Giannis Antetokounmpo_AdjUsage    6594.0
        79*34.2 + 79*33.4, # Stephen Curry_AdjUsage            6527.0
        75*35.7 + 75*35.9, # Damian Lillard_AdjUsage           6425.0
        81*32.9 + 81*29.7, # Andre Drummond_AdjUsage           6420.0
        79*32.2 + 76*33.8, # Isaiah Thomas_AdjUsage  
], index=top10.index)

In [148]:
top10 / top10_season_minutes

Russell Westbrook_AdjUsage           1.633601
James Harden_AdjUsage                1.358232
LeBron James_AdjUsage                1.369666
DeMarcus Cousins_AdjUsage         7347.000000
John Wall_AdjUsage                   1.251911
Karl-Anthony Towns_AdjUsage          1.182573
Giannis Antetokounmpo_AdjUsage       1.169833
Stephen Curry_AdjUsage               1.222193
Damian Lillard_AdjUsage              1.196462
Andre Drummond_AdjUsage              1.266122
Isaiah Thomas_AdjUsage               1.237140
dtype: float64

In [152]:
df.head()

,SCORE,HOMEDESCRIPTION,VISITORDESCRIPTION,EVENTNUM,GAME_ID,PCTIMESTRING,PERIOD,PLAYER1_ID,PLAYER1_NAME,PLAYER2_ID,PLAYER2_NAME,PLAYER3_ID,PLAYER3_NAME,SCOREMARGIN,Foul,Fouler,Fouler_team,is_home_team,Foulee,Foulee_team,PLAYER1_TEAM_NICKNAME,PLAYER2_TEAM_NICKNAME,PLAYER3_TEAM_NICKNAME,Home_score,Away_score,Minute,Second,Home_team,Away_team,Score_label
0,NaN,NaN,NaN,0,21500758,12:00,1,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,12,0,Jazz,Bucks,Bucks84_Jazz81
1,NaN,Jump Ball Gobert vs. Monroe: Tip to Middleton,NaN,1,21500758,12:00,1,203497,Rudy Gobert,202328,Greg Monroe,203114,Khris Middleton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,Bucks,Bucks,0,0,12,0,Jazz,Bucks,Bucks84_Jazz81
2,NaN,NaN,MISS Parker 17' Pullup Jump Shot,2,21500758,11:45,1,203953,Jabari Parker,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bucks,NaN,NaN,0,0,11,45,Jazz,Bucks,Bucks84_Jazz81
3,NaN,Neto REBOUND (Off:0 Def:1),NaN,3,21500758,11:44,1,203526,Raul Neto,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,NaN,NaN,0,0,11,44,Jazz,Bucks,Bucks84_Jazz81
4,0 - 2,Neto 4' Fadeaway Jumper (2 PTS),NaN,4,21500758,11:19,1,203526,Raul Neto,0,NaN,0,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,Jazz,NaN,NaN,0,2,11,19,Jazz,Bucks,Bucks84_Jazz81


In [348]:
usage_types1 = ['Layup', 'Dunk', 'Driving'] # Common situations to foul, x2 weight
usage_types2 = ['Shot', 'shot', 'Jumper', 'Turnover', 'REBOUND']
usage_type3 = 'AST' # only type which co-occurs during another event type

def make_usage_l2m(df, df_usage, l2m_games):
    
    to_drop = df[(df['PERIOD']==1) | (df['PERIOD']==2) | (df['PERIOD']==3)].index
    df_temp = df.drop(to_drop, axis=0)
    to_drop = df_temp[(df_temp['PERIOD']==4) & (df_temp['Minute']<2)].index
    df_temp = df_temp.drop(to_drop, axis=0)
    mask = df_temp['GAME_ID'].isin(l2m_games)
    df_temp = df_temp[mask]
    
    c = 0
    for idx, row in df_temp.iterrows():
        c += 1
        if c % 50000 == 0: print(c, ' rows processed')

        if type(row['Foul']) == float:
        
            usage1done = False
            usage2done = False
            if type(row['VISITORDESCRIPTION']) == str: 
                text = row['VISITORDESCRIPTION']
            elif (type(row['HOMEDESCRIPTION']) == str):
                text = row['HOMEDESCRIPTION']
            else:
                text = ''
                usage1done = True
                usage2done = True                
# Not using home versus away splits, because there is too low a total count of fouls
# Biggest hypothesized factor in foul bias would be strength of opponent, and can't quantify
# that accurately (ex: strength of schedule) given labeled data restricted to last 2 minutes
#                 is_home_text = 'error'
#                 player2text = 'error'
            
            for usage in usage_types1:
                if (usage in text) & (not usage1done):
                    usage1done = True
                    plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_AdjUsage')
                    # Plus 1 twice due to double weight for driving shots
                    plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_AdjUsage')

            if 'AST' in text:
                plus1(df_usage, row['GAME_ID'], str(row['PLAYER2_NAME']) + '_AdjUsage')
                
            if not usage1done:
                for usage in usage_types2:
                    if (usage in text) & (not usage2done):
                        usage2done = True
                        plus1(df_usage, row['GAME_ID'], str(row['PLAYER1_NAME']) + '_AdjUsage')

        elif type(row['Foul']) == str:
            if row['Foul'] in foul_types_dict.keys():
                foul = foul_types_dict[row['Foul']]
                if foul in ['Shooting']:
                    plus1(df_usage, row['GAME_ID'], str(row['Foulee']) + '_' + foul)
                    plus1(df_usage, row['GAME_ID'], str(row['Foulee_team']) + '_' + foul)
                    plus1(df_usage, row['GAME_ID'], str(row['Fouler_team']) + '_'+foul+'_given')
                    plus1(df_usage, row['GAME_ID'], str(row['Foulee']) + '_AdjUsage')
                elif foul in ['Offensive']:
                    plus1(df_usage, row['GAME_ID'], str(row['Fouler']) + '_' + foul)
                    plus1(df_usage, row['GAME_ID'], str(row['Fouler_team']) + '_' + foul)
                    plus1(df_usage, row['GAME_ID'], str(row['Foulee_team']) + '_'+foul+'_taken')
                    plus1(df_usage, row['GAME_ID'], str(row['Fouler']) + '_AdjUsage')

In [ ]:
def l2m_home_away(score_text):
    try:
        t = score_text.split('_')
        team1 = t[0]
        team2 = t[1].split('(')[0] # to get rid of overtime label if it has it, if not still works
        return team1 + '_' + team2
    except: return 'error'

df_l2m['Score_label'] = df_l2m['Score'].map(l2m_home_away)

In [335]:
# Finally switching to game_id, instead of score_label!
l2m_games = set(df_l2m['game_id'])

In [349]:
l2m_usage_pbp = pd.DataFrame()

make_usage_l2m(df, l2m_usage_pbp, l2m_games)

In [339]:
to_drop = df[(df['PERIOD']==1) | (df['PERIOD']==2) | (df['PERIOD']==3)].index
df_temp = df.drop(to_drop, axis=0)
to_drop = df_temp[(df_temp['PERIOD']==4) & (df_temp['Minute']<2)].index
df_temp = df_temp.drop(to_drop, axis=0)
mask = df_temp['GAME_ID'].isin(l2m_games)
len(df_temp[mask]['GAME_ID'].unique())

435

In [340]:
set1 = set(df_temp['GAME_ID'])
len(set1)

2460

In [341]:
len(l2m_games)

439

In [342]:
len(set1.intersection(l2m_games))
# It is 99% working now!

435

In [343]:
l2m_games.difference(set1) # Will exclude these 4 games

{nan, nan, nan, nan}

In [350]:
l2m_usage_pbp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 435 entries, 21500758 to 21600289
Columns: 1159 entries, Greg Monroe_AdjUsage to Nicolas Laprovittola_AdjUsage
dtypes: float64(1159)
memory usage: 3.9 MB


In [351]:
l2m_usage_pbp.filter(regex='OKC.*').sum()

OKC_Shooting           81.0
OKC_Offensive_taken     9.0
OKC_Shooting_given     89.0
OKC_Offensive           6.0
dtype: float64

In [359]:
l2m_usage_pbp.filter(regex='Andre Drummond.*').sum() 

Andre Drummond_AdjUsage     143.0
Andre Drummond_Offensive      2.0
Andre Drummond_Shooting      15.0
dtype: float64

In [358]:
l2m_usage_pbp.filter(regex='Stephen Curry.*').sum() 

Stephen Curry_AdjUsage    119.0
Stephen Curry_Shooting      8.0
dtype: float64

In [354]:
l2m_usage_pbp.filter(regex='HOU.*').sum()

HOU_Shooting_given     75.0
HOU_Shooting           80.0
HOU_Offensive           4.0
HOU_Offensive_taken    17.0
dtype: float64

In [360]:
top10_l2m = l2m_usage_pbp.filter(regex='.*_AdjUsage').sum().sort_values(ascending=False).head(22)
top10_l2m

Russell Westbrook_AdjUsage        427.0
DeMarcus Cousins_AdjUsage         357.0
James Harden_AdjUsage             327.0
Karl-Anthony Towns_AdjUsage       307.0
LeBron James_AdjUsage             292.0
Kemba Walker_AdjUsage             290.0
Damian Lillard_AdjUsage           285.0
Isaiah Thomas_AdjUsage            283.0
CJ McCollum_AdjUsage              278.0
Zach Randolph_AdjUsage            271.0
John Wall_AdjUsage                256.0
Anthony Davis_AdjUsage            251.0
Dwyane Wade_AdjUsage              232.0
Mike Conley_AdjUsage              226.0
Bradley Beal_AdjUsage             224.0
Kawhi Leonard_AdjUsage            224.0
Paul George_AdjUsage              223.0
Marc Gasol_AdjUsage               219.0
Andrew Wiggins_AdjUsage           219.0
Kyle Lowry_AdjUsage               214.0
Enes Kanter_AdjUsage              204.0
Giannis Antetokounmpo_AdjUsage    202.0
dtype: float64

In [361]:
top10_l2m = pd.Series(
    data=[
        427, # Russell Westbrook_AdjUsage        9074.0
        327, # James Harden_AdjUsage             8248.0
        292, # LeBron James_AdjUsage             7537.0
        1, # DeMarcus Cousins_AdjUsage         7347.0
        256, # John Wall_AdjUsage                7044.0
        307, # Karl-Anthony Towns_AdjUsage       6691.0
        202, # Giannis Antetokounmpo_AdjUsage    6594.0
        119, # Stephen Curry_AdjUsage            6527.0
        285, # Damian Lillard_AdjUsage           6425.0
        143, # Andre Drummond_AdjUsage           6420.0
        283 # Isaiah Thomas_AdjUsage            6325.0
], index=top10.index)

top10_l2m / top10_l2m_minutes

Russell Westbrook_AdjUsage        Total_l2m_minutes        8.895833
Win_percent_...
James Harden_AdjUsage             Total_l2m_minutes        5.542373
Win_percent_...
LeBron James_AdjUsage             Total_l2m_minutes        5.725490
Win_percent_...
DeMarcus Cousins_AdjUsage                                                         1
John Wall_AdjUsage                Total_l2m_minutes        3.878788
Win_percent_...
Karl-Anthony Towns_AdjUsage       Total_l2m_minutes        8.527778
Win_percent_...
Giannis Antetokounmpo_AdjUsage    Total_l2m_minutes        6.121212
Win_percent_...
Stephen Curry_AdjUsage            Total_l2m_minutes        4.407407
Win_percent_...
Damian Lillard_AdjUsage           Total_l2m_minutes        7.500000
Win_percent_...
Andre Drummond_AdjUsage           Total_l2m_minutes        4.085714
Win_percent_...
Isaiah Thomas_AdjUsage            Total_l2m_minutes        5.895833
Win_percent_...
dtype: object

In [362]:
top10 / top10_season_minutes

Russell Westbrook_AdjUsage           1.633601
James Harden_AdjUsage                1.358232
LeBron James_AdjUsage                1.369666
DeMarcus Cousins_AdjUsage         7347.000000
John Wall_AdjUsage                   1.251911
Karl-Anthony Towns_AdjUsage          1.182573
Giannis Antetokounmpo_AdjUsage       1.169833
Stephen Curry_AdjUsage               1.222193
Damian Lillard_AdjUsage              1.196462
Andre Drummond_AdjUsage              1.266122
Isaiah Thomas_AdjUsage               1.237140
dtype: float64

In [364]:
top10_l2m_minutes

Russell Westbrook_AdjUsage        Total_l2m_minutes      48.000000
Win_percent_2...
James Harden_AdjUsage             Total_l2m_minutes      59.000000
Win_percent_2...
LeBron James_AdjUsage             Total_l2m_minutes      51.000000
Win_percent_2...
DeMarcus Cousins_AdjUsage                                                         1
John Wall_AdjUsage                Total_l2m_minutes      66.000000
Win_percent_2...
Karl-Anthony Towns_AdjUsage       Total_l2m_minutes      36.000000
Win_percent_2...
Giannis Antetokounmpo_AdjUsage    Total_l2m_minutes      33.000000
Win_percent_2...
Stephen Curry_AdjUsage            Total_l2m_minutes      27.000000
Win_percent_2...
Damian Lillard_AdjUsage           Total_l2m_minutes      38.000000
Win_percent_2...
Andre Drummond_AdjUsage           Total_l2m_minutes      35.000000
Win_percent_2...
Isaiah Thomas_AdjUsage            Total_l2m_minutes      48.000000
Win_percent_2...
dtype: object

In [385]:
print(25/48*36)
l2m_usage_pbp.filter(regex='Russell Westbrook.*').sum() 

18.75


Russell Westbrook_AdjUsage     427.0
Russell Westbrook_Shooting      25.0
Russell Westbrook_Offensive      1.0
dtype: float64

In [386]:
print(33/59*36)
l2m_usage_pbp.filter(regex='James Harden.*').sum() 

20.135593220338983


James Harden_AdjUsage     327.0
James Harden_Shooting      33.0
James Harden_Offensive      1.0
dtype: float64

In [387]:
print(25/51*36)
l2m_usage_pbp.filter(regex='LeBron James.*').sum() 

17.64705882352941


LeBron James_Shooting      25.0
LeBron James_AdjUsage     292.0
LeBron James_Offensive      2.0
dtype: float64

In [388]:
print(22/33*36)
l2m_usage_pbp.filter(regex='Giannis Antetokounmpo.*').sum() 

24.0


Giannis Antetokounmpo_AdjUsage    202.0
Giannis Antetokounmpo_Shooting     22.0
dtype: float64

In [389]:
print(15/35*36)
l2m_usage_pbp.filter(regex='Andre Drummond.*').sum() 

15.428571428571427


Andre Drummond_AdjUsage     143.0
Andre Drummond_Offensive      2.0
Andre Drummond_Shooting      15.0
dtype: float64

In [378]:
top10_season_minutes

Russell Westbrook_AdjUsage        5554.6
James Harden_AdjUsage             6072.6
LeBron James_AdjUsage             5502.8
DeMarcus Cousins_AdjUsage            1.0
John Wall_AdjUsage                5626.6
Karl-Anthony Towns_AdjUsage       5658.0
Giannis Antetokounmpo_AdjUsage    5636.7
Stephen Curry_AdjUsage            5340.4
Damian Lillard_AdjUsage           5370.0
Andre Drummond_AdjUsage           5070.6
Isaiah Thomas_AdjUsage            5112.6
dtype: float64

In [380]:
print(602/5555*36)
df_usage.filter(regex='Russell Westbrook.*').sum() 

3.9013501350135016


Russell Westbrook_AdjUsage      9074.0
Russell Westbrook_<5ft           916.0
Russell Westbrook_Shooting       602.0
Russell Westbrook_Offensive       21.0
Russell Westbrook_Personal       327.0
Russell Westbrook_Traveling       13.0
Russell Westbrook_5-10ft          20.0
Russell Westbrook_Loose Ball      12.0
Russell Westbrook_15-22ft          1.0
Russell Westbrook_10-15ft          2.0
dtype: float64

In [381]:
print(754/6073*36)
df_usage.filter(regex='James Harden.*').sum() 

4.469619627861024


James Harden_AdjUsage      8248.0
James Harden_<5ft           754.0
James Harden_Shooting       649.0
James Harden_Personal       337.0
James Harden_Offensive       37.0
James Harden_Traveling       21.0
James Harden_Loose Ball      25.0
James Harden_10-15ft         16.0
James Harden_5-10ft          61.0
dtype: float64

In [382]:
print(488/5503*36)
df_usage.filter(regex='LeBron James.*').sum() 

3.192440487007087


LeBron James_AdjUsage      7537.0
LeBron James_Shooting       488.0
LeBron James_<5ft           883.0
LeBron James_Traveling       19.0
LeBron James_Personal       308.0
LeBron James_Offensive       36.0
LeBron James_5-10ft          34.0
LeBron James_Loose Ball      17.0
LeBron James_15-22ft          2.0
LeBron James_10-15ft          6.0
dtype: float64

In [383]:
print(470/5637*36)
df_usage.filter(regex='Giannis Antetokounmpo.*').sum() 

3.001596593932943


Giannis Antetokounmpo_AdjUsage      6594.0
Giannis Antetokounmpo_<5ft           685.0
Giannis Antetokounmpo_Shooting       470.0
Giannis Antetokounmpo_Personal       289.0
Giannis Antetokounmpo_Offensive       40.0
Giannis Antetokounmpo_Traveling       50.0
Giannis Antetokounmpo_Loose Ball      25.0
Giannis Antetokounmpo_5-10ft          19.0
Giannis Antetokounmpo_10-15ft          1.0
dtype: float64

In [384]:
print(358/5070*36)
df_usage.filter(regex='Andre Drummond.*').sum() 

2.542011834319527


Andre Drummond_AdjUsage      6420.0
Andre Drummond_Shooting       358.0
Andre Drummond_Personal       258.0
Andre Drummond_<5ft           625.0
Andre Drummond_Offensive       53.0
Andre Drummond_Loose Ball      62.0
Andre Drummond_Traveling       17.0
Andre Drummond_5-10ft          15.0
Andre Drummond_10-15ft          1.0
dtype: float64

In [390]:
df_usage_l2m.filter(regex='Andre Drummond.*').sum() 

2.542011834319527


Andre Drummond_Offensive_CNC    3.0
Andre Drummond_Shooting_CNC     1.0
Andre Drummond_Shooting_CC      1.0
Andre Drummond_Offensive_INC    1.0
dtype: float64

In [391]:
df_usage_l2m.filter(regex='Giannis Antetokounmpo.*').sum() 

Giannis Antetokounmpo_Shooting_INC     3.0
Giannis Antetokounmpo_Offensive_INC    1.0
Giannis Antetokounmpo_Shooting_CNC     9.0
Giannis Antetokounmpo_Shooting_CC      5.0
Giannis Antetokounmpo_Offensive_CNC    3.0
Giannis Antetokounmpo_Offensive_CC     1.0
dtype: float64

In [397]:
df_usage_l2m.filter(regex='LeBron James.*').sum() 

LeBron James_Shooting_CNC     15.0
LeBron James_Offensive_CNC     4.0
LeBron James_Shooting_CC       7.0
LeBron James_Shooting_INC      2.0
LeBron James_Offensive_CC      1.0
dtype: float64

In [396]:
df_usage_l2m.filter(regex='James Harden.*').sum() 

James Harden_Shooting_CNC     31.0
James Harden_Offensive_CC      4.0
James Harden_Shooting_CC      17.0
James Harden_Offensive_CNC     8.0
James Harden_Shooting_INC      2.0
James Harden_Offensive_INC     1.0
dtype: float64

In [395]:
df_usage_l2m.filter(regex='Russell Westbrook.*').sum() 

Russell Westbrook_Shooting_CC      15.0
Russell Westbrook_Shooting_CNC     35.0
Russell Westbrook_Offensive_CNC     5.0
Russell Westbrook_Shooting_INC      4.0
Russell Westbrook_Offensive_INC     1.0
dtype: float64

### Analyzing by Team, and by Brackets of High/Medium/Low Win Percent Teams

In [43]:
df_teams['Win_percent_2015_16'] = [
    10/82, 33/82, 42/82, 57/82, 48/82, 53/82, 42/82, 48/82, 48/82, 48/82, 40/82, 33/82, 32/82, 17/82,
    35/82, 42/82, 21/82, 33/82, 45/82, 30/82, 44/82, 56/82, 41/82, 67/82, 23/82, 55/82, 29/82,
    44/82, 73/82, 41/82
]

df_teams['Win_percent_2016_17'] = [
    28/82, 42/82, 41/82, 51/82, 53/82, 51/82, 43/82, 43/82, 41/82, 36/82, 51/82, 32/82, 31/82, 26/82,
    29/82, 33/82, 20/82, 40/82, 42/82, 34/82, 37/82, 51/82, 55/82, 61/82, 24/82, 47/82, 31/82,
    41/82, 67/82, 49/82
]

df_teams['Win_percent'] = (df_teams['Win_percent_2015_16'] + df_teams['Win_percent_2016_17'])/2
df_teams

,Total_l2m_minutes,Win_percent_2015_16,Win_percent_2016_17,Win_percent
76ers,26.0,0.121951,0.341463,0.231707
Bucks,33.0,0.402439,0.512195,0.457317
Bulls,38.0,0.512195,0.500000,0.506098
Cavaliers,51.0,0.695122,0.621951,0.658537
Celtics,48.0,0.585366,0.646341,0.615854
Clippers,26.0,0.646341,0.621951,0.634146
Grizzlies,59.0,0.512195,0.524390,0.518293
Hawks,49.0,0.585366,0.524390,0.554878
Heat,39.0,0.585366,0.500000,0.542683
Hornets,40.0,0.585366,0.439024,0.512195


In [44]:
df_teams.describe()

,Total_l2m_minutes,Win_percent_2015_16,Win_percent_2016_17,Win_percent
count,30.000000,30.000000,30.000000,30.000000
mean,38.766667,0.500000,0.500000,0.500000
std,13.265676,0.169314,0.136440,0.147067
min,14.000000,0.121951,0.243902,0.231707
25%,29.000000,0.402439,0.393293,0.391768
50%,36.500000,0.512195,0.500000,0.515244
75%,48.000000,0.585366,0.615854,0.577744
max,66.000000,0.890244,0.817073,0.853659
